CSV & Excel files ---> Structured Data

In [1]:
import pandas as pd
import os

In [2]:
os.makedirs("data/structured_files", exist_ok=True)

In [4]:
#Create sample data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

#Save as CSV
df=pd.DataFrame(data)
df.to_csv('data/structured_files/products.csv', index=False)


In [6]:
#Save as Excel witht multiple shetets
with pd.ExcelWriter('data/structured_files/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)

    #Add another sheet
    summary_data={
        'Category':['Electronics', 'Accessories'],
        'Total_Items': [3,2],
        'Total_Value': [1894.43, 108.74]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

CSV Processing

In [7]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

In [9]:
#Method1: CSV loader--> each row becomes a document

csv_loader=CSVLoader(
    file_path='data/structured_files/products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter':',',
        'quotechar':'"',
    }
)

csv_docs=csv_loader.load()
print(csv_docs)
print(f"Loaded {len(csv_docs)}")
print(f"\nFirst Document")
print(f"Content: \n{csv_docs[0].page_content}")
print(f"Metadata: {csv_docs[0].metadata}")

[Document(metadata={'source': 'data/structured_files/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: Mechanical keyboard with RGB backlighting'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 3}, page_content='Product: Monitor\nCategory: Electronics\nPrice: 299.99\nStock: 75\nDescription: 27-inch 4K monitor with HDR support'), Document(metadata={'source': 'data/structured_files/products.csv', 'row': 4}, page_content='Product: Webcam\nCategory: Electro

In [18]:
from typing import List
from langchain_core.documents import Document

#Method2: Custom CSV processing for better control
def process_csv_intelligently(filepath:str)->List[Document]:
    """Process CSV with intelligent Document creation"""
    df = pd.read_csv(filepath)
    documents=[]

    for idx,row in df.iterrows():
        #Create structured content
        content=f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}"""

        ###Custom Metadata creation
        doc=Document(
            page_content=content,
            metada={
                'source':filepath,
                'row_index':idx,
                'product_name':row['Product'],
                'Category':row['Category'],
                'Price':row['Price']
            }
        )
        documents.append(doc)
    return documents


In [19]:
pop=process_csv_intelligently('data/structured_files/products.csv')
print(pop)

[Document(metadata={}, page_content='Product Information:\n        Name: Laptop\n        Category: Electronics\n        Price: $999.99'), Document(metadata={}, page_content='Product Information:\n        Name: Mouse\n        Category: Accessories\n        Price: $29.99'), Document(metadata={}, page_content='Product Information:\n        Name: Keyboard\n        Category: Accessories\n        Price: $79.99'), Document(metadata={}, page_content='Product Information:\n        Name: Monitor\n        Category: Electronics\n        Price: $299.99'), Document(metadata={}, page_content='Product Information:\n        Name: Webcam\n        Category: Electronics\n        Price: $89.99')]


EXCEL PROCESSING

In [24]:
def process_excel_with_pandas(filepath: str)->List[Document]:
    """Process Excel with sheet awareness"""
    documents=[]
    #Read all sheets
    excel_file=pd.ExcelFile(filepath)

    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet_name)

        #Create a document for each sheet
        sheet_content=f"Sheet: {sheet_name}"
        sheet_content += f"Columns: {', '.join(df.columns)}\n"
        sheet_content += f"Rows: {len(df)}\n\n"
        sheet_content += df.to_string(index=False)

        doc = Document(
            page_content=sheet_content,
            metadata={
                'source':filepath,
                'sheet_name':sheet_name,
                'num_rows': len(df),
                'num_columns': len(df.columns)
            }
        )
        documents.append(doc)
    
    return documents

In [25]:
excel_docs=process_excel_with_pandas('data/structured_files/inventory.xlsx')
print(f"Processed_excel: {len(excel_docs)}")

Processed_excel: 2


In [26]:
excel_docs

[Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Products', 'num_rows': 5, 'num_columns': 5}, page_content='Sheet: ProductsColumns: Product, Category, Price, Stock, Description\nRows: 5\n\n Product    Category  Price  Stock                                         Description\n  Laptop Electronics 999.99     50 High-performance laptop with 16GB RAM and 512GB SSD\n   Mouse Accessories  29.99    200        Wireless optical mouse with ergonomic design\nKeyboard Accessories  79.99    150           Mechanical keyboard with RGB backlighting\n Monitor Electronics 299.99     75                 27-inch 4K monitor with HDR support\n  Webcam Electronics  89.99    100                1080p webcam with noise cancellation'),
 Document(metadata={'source': 'data/structured_files/inventory.xlsx', 'sheet_name': 'Summary', 'num_rows': 2, 'num_columns': 3}, page_content='Sheet: SummaryColumns: Category, Total_Items, Total_Value\nRows: 2\n\n   Category  Total_Items  Total_